In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
import torch.optim

# CODE FILES HERE
from models.vae.vae import Encoder, Decoder, Vae, PATH
from solver import Solver
from dataloader import DataLoader
from plot import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space, plot_latent_space_no_labels, \
plot_latent_manifold, plot_faces_grid, plot_faces_samples_grid

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cpu



In [3]:
# Choose the dataset and tune hyperparameters here!
dataset = "FF"

batch_size = 128
optimizer = torch.optim.Adam

if dataset == "MNIST":
    epochs = 100
    hidden_dim = 500
    z_dim = 20
    warmup_epochs = 0
    beta = 10 if z_dim == 2 else 10
    step_lr = False
    step_config = {
        "step_size" : 200,
        "gamma" : 0.1 # or 0.75
    }
    optim_config = {
        "lr": 1e-3,
        "weight_decay": 1e-4
    }
elif dataset == "LFW":
    epochs = 500
    hidden_dim = 700
    z_dim = 20
    warmup_epochs = 0
    beta = 10 if z_dim == 2 else 10
    step_lr = True
    step_config = {
        "step_size" : 30,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-1,
        "weight_decay": 1e-4
    }
elif dataset == "FF":
    epochs = 100
    hidden_dim = 200
    z_dim = 2
    warmup_epochs = 0
    beta = 1
    step_lr = True
    step_config = {
        "step_size" : 300,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-2,
        "weight_decay": 1e-4
    }

In [4]:
data_loader = DataLoader(PATH, batch_size, dataset, z_dim)
model = Vae(data_loader.input_dim, hidden_dim, z_dim)

In [9]:
solver = Solver(model, data_loader, optimizer, z_dim, epochs, step_lr, step_config, optim_config, warmup_epochs, beta)
solver.main()

+++++ START RUN +++++
====> Epoch: 1 train set loss avg: 380.5816
====> Test set loss avg: 431.2410
0.5534181594848633 seconds for epoch 1
====> Epoch: 2 train set loss avg: 370.4115
====> Test set loss avg: 379.3982
0.6061704158782959 seconds for epoch 2
====> Epoch: 3 train set loss avg: 364.1240
====> Test set loss avg: 370.6210
0.6168358325958252 seconds for epoch 3
====> Epoch: 4 train set loss avg: 359.7450
====> Test set loss avg: 361.9076
0.6620774269104004 seconds for epoch 4
====> Epoch: 5 train set loss avg: 356.6546
====> Test set loss avg: 363.8974
0.49134135246276855 seconds for epoch 5
====> Epoch: 6 train set loss avg: 354.5638
====> Test set loss avg: 357.1764
0.4749746322631836 seconds for epoch 6
====> Epoch: 7 train set loss avg: 353.0324
====> Test set loss avg: 359.7890
0.475970983505249 seconds for epoch 7
====> Epoch: 8 train set loss avg: 351.9313
====> Test set loss avg: 355.1399
0.5508201122283936 seconds for epoch 8
====> Epoch: 9 train set loss avg: 350.956

KeyboardInterrupt: 

In [ ]:
# Insert name of model here if want to load a model, e.g. solver.save_model_dir + "/VAE_MNIST_train_loss=151.39_z=2.pt"
#solver = torch.load(solver.save_model_dir + "/VAE_MNIST_train_loss=97.15_z=20.pt")
#solver.model.eval()

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, 4)

In [ ]:
plot_gaussian_distributions(solver)

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 0
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
                             solver.train_loss_history["train_loss_acc"], \
                             solver.test_loss_history, \
                             solver.train_loss_history["recon_loss_acc"], \
                             solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
        print("overfitting: {:.2f}".format(abs(test_loss-train_loss)))
plot_rl_kl(solver, 4)

In [ ]:
# visualize q(z) (latent space z)
if solver.z_dim == 2:
    if solver.loader.dataset == "FF":
        plot_latent_space_no_labels(solver)
    else:
        plot_latent_space(solver)
else:
    print("Plot of latent space not possible as dimension of z is not 2")

In [ ]:
# Visualizations of learned data manifold for generative models with two-dimensional latent space
if solver.z_dim == 2:
    if solver.loader.dataset == "MNIST":
        plot_latent_manifold(solver, "bone")
    if solver.loader.dataset == "LFW" or solver.loader.dataset == "FF":
        plot_latent_manifold(solver, "gray", n=10, fig_size=(10, 8))
else:
    print("Plot is not possible as dimension of z is not 2")

In [ ]:
# plots real faces and in grid samples
if solver.loader.dataset == "LFW" or solver.loader.dataset == "FF":
    plot_faces_grid(225, 15, solver)
    plot_faces_samples_grid(225, 15, solver)

In [ ]:
last_train_loss = solver.train_loss_history["train_loss_acc"][-1]
torch.save(solver, solver.save_model_dir + "/VAE_" + solver.loader.dataset + "_train_loss=" + "{0:.2f}".format(last_train_loss) + "_z=" + str(solver.z_dim) + ".pt")